# Using ProcessorScriptDocker

In [ ]:
# Define now the host path maps to the docker container path.
#This must match the docker command line to start the container
path_map = {}
path_map[r"C:\Users\Andres\source\repos"] = '/shared'

Start the CASM docker container with a command like (the path for the mapping of filesystem must match to the one set above):

    docker run -P --rm -it -v c:\Users\Andres\source\repos:/shared casmcode/casm bash 

In [ ]:
!docker ps

In [ ]:
from tinc import *

We will use the ```ProcessorScriptDocker``` class to execute on demand computation on the container.

In [ ]:
proc = ProcessorScriptDocker("test")
# set the container to run on:
proc.set_container_id(proc.find_container_id("casmcode/casm"))
# set the command line to run:
proc.set_command_line("ls /")
proc.capture_output(r"C:\Users\Andres\source\repos\test.txt") # Store the output of stdout to a file
# Execute
# proc.debug = True # This will give more detailed information of what's happening
proc.process()

In [ ]:
!cat "C:/Users/Andres/source/repos/test.txt"

You can create a parameter that will trigger execution on docker whenever it changes:

In [ ]:
path = ParameterString("path")
proc.register_parameter(path)

You can also make the command line depend on the parameter value by passing the parameter name surrounded by '%%' to the argument template:

In [ ]:
proc.set_argument_template("%%path%%")

Whenever you change the value, the command will be run in the docker container:

In [ ]:
path.interactive_widget()

Finally if you want something to happen when the processor is done, you can register a 'done_callback' or you can use a DiskBuffer for the output

In [ ]:
def done_callback(processor, ok):
    if ok:
        print(f" --- Done writing to: {processor.output_files[0]}")
    else:
        print(" --- Something went wrong")

proc.done_callback = done_callback

In [ ]:
path.value = "/usr/lib"

In [ ]:
path.value = "/usr/error"

Now using a DiskBuffer:

In [ ]:
#proc.debug = True
proc.done_callback = None
db = DiskBufferText("ls_text", "db_out.txt", '', "C:/Users/Andres/source/repos/" )
proc.output_files = [db]

In [ ]:
path.value = "/usr/local"

You can assign a callback to the disk buffer, or use it as an interactive widget in ipython if the disk buffer type supports it.

In [ ]:
from ipywidgets import VBox

v = VBox([path.interactive_widget(), db.interactive_widget()])
v

## CASM docker container tools

Create parameters:

In [ ]:
settings_file_name = ParameterString("settings_file_name", default_value="genetic_alg_settings.json")
hall_of_fame_index = ParameterInt("hall_of_fame_index", default_value=0)
fit_dir = ParameterString("fit_dir", default_value='')

These parameters will trigger computation on the docker container whenever they change. 

In [ ]:
proc = ProcessorScriptDocker("check")
# names within '%%' that match parameter names will be replaced by their value
proc.set_command_line("casm-learn -s %%settings_file_name%% --checkhull --indiv %%hall_of_fame_index%%")
proc.set_container_id(proc.find_container_id("casmcode/casm"))
# Capture the std output of the command to file
proc.capture_output()
# Register paramters with processor. Changes trigger computation
proc.register_parameter(hall_of_fame_index)
proc.register_parameter(settings_file_name)

# The output will be managed by a diskbuffer, to update data everywhere
db = DiskBufferText("check_buffer", "check.0", "out/", "/shared")
#tclient.register_disk_buffer(db)
# You can set a disk buffer to be the output of a Processor
proc.output_files = [db]

# Because we want to change directory and output file on every run,
# We define a 'prepare' function
def prepare_check(p):
    p.running_dir = fit_dir.value
    db.set_base_filename(f"check.{hall_of_fame_index.value}")
    print(f"Set output to: {p.output_files[0]}")
    #print(p._get_arguments())
    return True
# This function will be called right before calling the command for ProcessorScript
proc.prepare = prepare_check
#proc.debug = True

In [ ]:
# Processor 2
proc2 = ProcessorScriptDocker("select")
proc2.set_container_id(proc.find_container_id("casmcode/casm"))
proc2.set_command_line("casm-learn-s %%settings_file_name%% --select %%hall_of_fame_index%%")
proc2.capture_output()
db2 = DiskBufferText("select_buffer","select_fit_eci.out", "out/", "/shared")
#tclient.register_disk_buffer(db2)
proc2.register_parameter(hall_of_fame_index)
proc2.register_parameter(settings_file_name)

def prepare_select(p):
    p.running_dir = fit_dir.value
    return True

proc2.prepare = prepare_select
#proc2.debug = True

In [ ]:
# Processor 3
proc3 = ProcessorScriptDocker("fit_eci")
proc3.set_command_line("casm query -k comp formation_energy hull_dist clex clex_hulldist -o %%:OUTFILE:0%%");
proc3.set_container_id(proc3.find_container_id("casmcode/casm"))
db3 = DiskBufferText("fit_eci_buffer","select_fit_eci.out", "out/", "/shared")
#tclient.register_disk_buffer(db3)
proc3.output_files = [db]
proc3.register_parameter(hall_of_fame_index)
proc3.register_parameter(settings_file_name)

def prepare_fit(p):
    p.running_dir = fit_dir.value
    return True
proc3.prepare = prepare_fit
#proc3.debug = True

In [ ]:
from ipywidgets import VBox

v = VBox([settings_file_name.interactive_widget(), hall_of_fame_index.interactive_widget(), fit_dir.interactive_widget() ])
v